# Iterative modeling process
Importing our functions, loading in our images, train test splitting and one hot encoding them

In [1]:
from google.colab import drive
drive.mount('/content/drive')


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
cd /content/drive/MyDrive/ColabNotebooks/Emoition-classification-from-facial-expression

In [ ]:
pip install keract


In [ ]:
from Functions import *
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox


In [ ]:
filname = 'fer2013.csv'
label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
names=['emotion','pixels','usage']
df=pd.read_csv('fer2013.csv',names=names)
im=df['pixels']
df.head(10)
X, Y = getData(filname)
num_class = len(set(Y))
print(num_class)
N, D = X.shape
X = X.reshape(N, 48, 48, 1)
cascade = load_cascade_classifier_xml()
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
y_train = (np.arange(num_class) == y_train[:, None]).astype(np.float32)
y_test = (np.arange(num_class) == y_test[:, None]).astype(np.float32)
Y = (np.arange(num_class) == Y[:, None]).astype(np.float32)



In [ ]:
picdict = pkl.load(open('picdict2.pkl','rb'))
visx = []
visy = []
for label, pic in picdict.items():
  visx.append(pic)
  visy.append(label)
  

The first model that I started out with was pretrained and upon looking at its activations it seemed to not detect the characteristics of the expressions it was predicting. 

In [ ]:
model = Sequential()
input_shape = (48,48,1)
model.add(Conv2D(64, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(7))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')

In [ ]:
model.load_weights('firstmodel')

In [ ]:
# model.save_weights('firstmodel')

In [ ]:


model_eval(model,X_test,y_test,label_map,visx,visy)


#### Second Model: Make layer simpler to see if it improves my model

In [ ]:
model1 = Sequential()
input_shape = (48,48,1)
model1.add(Conv2D(512, (7, 7), input_shape=input_shape,activation='relu', padding='same'))
model1.add(Conv2D(512, (7, 7), input_shape=input_shape,activation='relu', padding='same'))

model1.add(BatchNormalization())
model1.add(MaxPooling2D(pool_size=(2, 2)))



model1.add(Flatten())
model1.add(Activation('relu'))
model1.add(Dense(64))
model1.add(Dense(7))
model1.add(Activation('softmax'))

model1.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')




In [ ]:
model1.fit(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

In [ ]:
# model1.save_weights('model1')
model1.load_weights('model1')

In [ ]:
model_eval(model1,X_test,y_test,label_map,visx,visy)


Happiness predicted well, surprise predictd well, rest are all pretty low. 

## Third Model: Let's see if we can make another layer 

In [ ]:
model2 = Sequential()
input_shape = (48,48,1)
model2.add(Conv2D(64, (3, 3), input_shape=input_shape,activation='relu', padding='same'))
model2.add(MaxPooling2D(pool_size=(3, 3)))


model2.add(Conv2D(64, (5, 5),activation='relu',padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))



model2.add(Flatten())



model2.add(Activation('relu'))
model2.add(Dense(64))
model2.add(Activation('relu'))
model2.add(Dense(7))
model2.add(Activation('softmax'))

model2.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')

In [ ]:
model2.fit(X_train,y_train,epochs=15,validation_data=(X_test,y_test))

In [ ]:
# model2.save_weights('model2')
model2.load_weights('model2')

model_eval(model2,X_test,y_test,label_map,visx,visy)


Happiness still predicted well but not as well so our prediction decreased than the previous model. 

In [ ]:
model3 = Sequential()
input_shape = (48,48,1)
model3.add(Conv2D(256, (2, 2),activation='relu',padding='same'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Conv2D(256, (2, 2),activation='relu',padding='same'))
model3.add(MaxPooling2D(pool_size=(2, 2)))


model3.add(Flatten())

model3.add(Dense(256))
model3.add(Activation('relu'))
model3.add(Dense(128))
model3.add(Activation('relu'))
model3.add(Dense(7))
model3.add(Activation('softmax'))

model3.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')

In [ ]:
model3.fit(X_train,y_train,epochs=10,validation_data=(X_test,y_test))



In [ ]:
model3.load_weights('model3')
# model3.save_weights('model3')
model_eval(model3,X_test,y_test,label_map,visx,visy)


Still do not have a big difference from other models. Let's keep trying

## Model 4:

In [ ]:
model4 = Sequential()
input_shape = (48,48,1)
model4.add(Conv2D(64, (2, 2), input_shape=input_shape,activation='relu', padding='same'))
model4.add(Conv2D(64, (2, 2), input_shape=input_shape,activation='relu', padding='same'))
model4.add(Conv2D(64, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
model4.add(BatchNormalization())
model4.add(MaxPooling2D(pool_size=(2, 2)))

model4.add(Conv2D(128, (2, 2),activation='relu',padding='same'))
model4.add(Conv2D(128, (4, 4),activation='relu',padding='same'))
model4.add(Conv2D(128, (2, 2),activation='relu',padding='same'))

model4.add(BatchNormalization())
model4.add(MaxPooling2D(pool_size=(2, 2)))


model4.add(Flatten())



model4.add(Activation('relu'))
model4.add(Dense(250))

model4.add(BatchNormalization())
model4.add(Activation('relu'))
model4.add(Dense(64))
model4.add(Activation('relu'))
model4.add(Dense(7))
model4.add(Activation('softmax'))

model4.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')


In [ ]:
model4.fit(X_train,y_train,epochs=10,validation_data=(X_test,y_test))


In [ ]:
# model4.save_weights('model4')
model4.load_weights('model4')


In [ ]:
model_eval(model4,X_test,y_test,label_map,visx,visy)


In [ ]:
model5 = Sequential()
input_shape = (48,48,1)
model5.add(Conv2D(512, (2, 2), input_shape=input_shape,activation='relu', padding='same'))
model5.add(Conv2D(256, (3, 3), input_shape=input_shape,activation='relu', padding='same'))
model5.add(Conv2D(128, (4, 4),activation='relu',padding='same'))
model5.add(Conv2D(64, (5, 5),activation='relu',padding='same'))
model5.add(BatchNormalization())
model5.add(MaxPooling2D(pool_size=(2, 2)))
model5.add(Conv2D(512, (2, 2), input_shape=input_shape,activation='relu', padding='same'))
model5.add(Conv2D(256, (3, 3), input_shape=input_shape,activation='relu', padding='same'))
model5.add(Conv2D(128, (4, 4),activation='relu',padding='same'))
model5.add(Conv2D(64, (5, 5),activation='relu',padding='same'))

model5.add(BatchNormalization())
model5.add(MaxPooling2D(pool_size=(2, 2)))


model5.add(Flatten())


model5.add(Dense(125))
model5.add(BatchNormalization())
model5.add(Activation('relu'))

model5.add(Dense(15))
model5.add(BatchNormalization())
model5.add(Activation('relu'))

model5.add(Dense(7))
model5.add(BatchNormalization())
model5.add(Activation('softmax'))

model5.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')


In [ ]:
model5.fit(X_train,y_train,epochs=10,validation_data=(X_test,y_test))


In [ ]:
# model5.save_weights('model5')
model5.load_weights('model5')

model_eval(model5,X_test,y_test,label_map,visx,visy)


In [ ]:
model6 = Sequential()
input_shape = (48,48,1)
model6.add(Conv2D(512, (2, 2), input_shape=input_shape,activation='relu', padding='same'))
model6.add(Conv2D(512, (3, 3), input_shape=input_shape,activation='relu', padding='same'))

model6.add(MaxPooling2D(pool_size=(2, 2)))
model6.add(Dropout(0.2))

model6.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
model6.add(MaxPooling2D(pool_size=(2, 2)))
model6.add(Dropout(0.2))

model6.add(Conv2D(512, (2, 2),activation='relu',padding='same'))
model6.add(BatchNormalization())
model6.add(MaxPooling2D(pool_size=(2, 2)))
model6.add(Dropout(0.2))

model6.add(Flatten())
model6.add(BatchNormalization())

model6.add(Dense(500))
model6.add(Activation('relu'))
model6.add(Dropout(0.5))

model6.add(Dense(500))
model6.add(Activation('relu'))
model6.add(Dropout(0.5))

model6.add(BatchNormalization())

model6.add(Dense(500))
model6.add(Activation('relu'))
model6.add(Dropout(0.3))

model6.add(BatchNormalization())


model6.add(Dense(125))
model6.add(Activation('relu'))
model6.add(Dropout(0.3))



model6.add(Dense(7))
model6.add(Activation('softmax'))

model6.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')


In [ ]:
model6.fit(X_train,y_train,epochs=15,batch_size=100,validation_data=(X_test,y_test))


In [ ]:
# model6.save_weights('model6')
model6.load_weights('model6')

model_eval(model6,X_test,y_test,label_map,visx,visy)


In [ ]:
model7 = Sequential()
input_shape = (48,48,1)
model7.add(Conv2D(512, (2, 2), input_shape=input_shape,activation='relu', padding='same'))

model7.add(MaxPooling2D(pool_size=(2, 2)))

model7.add(Conv2D(128, (2, 2), activation='relu', padding='same'))
model7.add(Dropout(0.2))
model7.add(Conv2D(128, (6, 6), activation='relu', padding='same'))
model7.add(Dropout(0.2))

model7.add(MaxPooling2D(pool_size=(2, 2)))

model7.add(Conv2D(128, (2, 2),activation='relu',padding='same'))
model7.add(Dropout(0.2))

model7.add(Conv2D(128, (4, 4), activation='relu', padding='same'))
model7.add(Dropout(0.2))
model7.add(BatchNormalization())


model7.add(Flatten())
model7.add(BatchNormalization())


model7.add(Dense(900))
model7.add(Activation('relu'))
model7.add(Dropout(0.2))

model7.add(BatchNormalization())

model7.add(Dense(500))
model7.add(Activation('relu'))
model7.add(Dropout(0.25))

model7.add(BatchNormalization())


model7.add(Dense(500))
model7.add(Activation('relu'))
model7.add(Dropout(0.2))

model7.add(BatchNormalization())

model7.add(Dense(500))
model7.add(Activation('relu'))
model7.add(Dropout(0.2))

model7.add(BatchNormalization())

model7.add(Dense(500))
model7.add(Activation('relu'))
model7.add(Dropout(0.2))

model7.add(BatchNormalization())

model7.add(Dense(500))
model7.add(Activation('relu'))
model7.add(Dropout(0.25))

model7.add(BatchNormalization())


model7.add(Dense(500))
model7.add(Activation('relu'))
model7.add(Dropout(0.2))

model7.add(BatchNormalization())

model7.add(Dense(100))
model7.add(Activation('relu'))
model7.add(Dropout(0.2))

model7.add(BatchNormalization())

model7.add(Dense(50))
model7.add(Activation('relu'))
model7.add(Dropout(0.1))

model7.add(BatchNormalization())

model7.add(Dense(25))
model7.add(Activation('relu'))
model7.add(Dropout(0.1))

model7.add(Dense(7))
model7.add(Activation('softmax'))

model7.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')

In [ ]:
model7.fit(X_train,y_train,epochs=15,validation_data=(X_test,y_test))


In [ ]:
model7.save_weights('model7')
# model7.load_weights('model7')

model_eval(model7,X_test,y_test,label_map,visx,visy)


In [ ]:
model8 = Sequential()
input_shape = (48,48,1)
model8.add(Conv2D(128, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
model8.add(Conv2D(128, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
model8.add(Conv2D(128, (5, 5), input_shape=input_shape,activation='relu', padding='same'))

model8.add(MaxPooling2D(pool_size=(2, 2)))

model8.add(Conv2D(256, (5, 5),activation='relu',padding='same'))
model8.add(Conv2D(256, (3, 3),activation='relu',padding='same'))

model8.add(MaxPooling2D(pool_size=(2, 2)))

model8.add(Conv2D(1024, (3, 3),activation='relu',padding='same'))
model8.add(BatchNormalization())
model8.add(MaxPooling2D(pool_size=(2, 2)))

model8.add(Flatten())
model8.add(BatchNormalization())


model8.add(Dense(999))
model8.add(Activation('relu'))
model8.add(Dropout(0.3))
model8.add(BatchNormalization())

model8.add(Dense(750))
model8.add(Activation('relu'))
model8.add(Dropout(0.3))

model8.add(BatchNormalization())


model8.add(Dense(500))
model8.add(Activation('relu'))
model8.add(Dropout(0.2))

model8.add(BatchNormalization())

model8.add(Dense(100))
model8.add(Activation('relu'))
model8.add(Dropout(0.2))

model8.add(BatchNormalization())

model8.add(Dense(25))
model8.add(Activation('relu'))
model8.add(Dropout(0.1))

model8.add(Dense(7))
model8.add(Activation('softmax'))

model8.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')

In [ ]:
model8.fit(X_train,y_train,epochs=15,validation_data=(X_test,y_test))


In [2]:
# model8.save_weights('model8')
model8.load_weights('model8')

model_eval(model8,X_test,y_test,label_map,visx,visy)


NameError: name 'model8' is not defined

In [ ]:

final_model = Sequential()
final_model = Sequential()
input_shape = (48,48,1)
final_model.add(Conv2D(256, (2, 2), input_shape=input_shape,activation='relu', padding='same'))
final_model.add(BatchNormalization())
final_model.add(Dropout(0.2))

final_model.add(Conv2D(256, (2, 2), input_shape=input_shape,activation='relu', padding='same'))
final_model.add(BatchNormalization())
final_model.add(Dropout(0.2))

final_model.add(Conv2D(256, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
final_model.add(Dropout(0.2))

final_model.add(BatchNormalization())
final_model.add(MaxPooling2D(pool_size=(2, 2)))

final_model.add(Conv2D(512, (2, 2),activation='relu',padding='same'))
final_model.add(Dropout(0.2))

final_model.add(BatchNormalization())

final_model.add(Conv2D(512, (4, 4),activation='relu',padding='same'))
final_model.add(Dropout(0.2))

final_model.add(BatchNormalization())

final_model.add(Conv2D(512, (2, 2),activation='relu',padding='same'))
final_model.add(Dropout(0.2))


final_model.add(BatchNormalization())
final_model.add(MaxPooling2D(pool_size=(2, 2)))


final_model.add(Flatten())
final_model.add(Dense(750))
final_model.add(Activation('relu'))
final_model.add(Dropout(0.3))

final_model.add(Dense(750))
final_model.add(Activation('relu'))
final_model.add(Dropout(0.3))

final_model.add(Dense(750))
final_model.add(Activation('relu'))
final_model.add(Dropout(0.3))

final_model.add(Activation('relu'))
final_model.add(Dense(250))
final_model.add(Dropout(0.2))

final_model.add(BatchNormalization())
final_model.add(Activation('relu'))
final_model.add(Dense(64))
final_model.add(Activation('relu'))
final_model.add(Dense(7))
final_model.add(Activation('softmax'))

final_model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')




In [ ]:
final_model.fit(X,Y,epochs=20,batch_size=100)


In [ ]:
final_model.save_weights('FinalModel')
model_eval(final_model,X_test,y_test,label_map,visx,visy)


In [ ]:
#test it on a picture and use visualiztaions to see what the model sees
#explain the problem with testing it on your self
#bring in haar cascade classifier
#show the haar cascade classifier finding your face and multiple faces
#use the cascade classifier to predict on your face
#use both to predict on you face and show the visualizations
#talk about context and body language
#bring in and demonstrate pose estimator
#show video of your program predicting in realtime
#discuss failures and next steps